In [1]:
from PIL import Image
import glob

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 128*128 사이즈로 바꾸어 저장합니다.
	target_size=(224,224)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [2]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\scissors"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
가위 이미지 resize 완료!


In [3]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
바위 이미지 resize 완료!


In [4]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

0  images to be resized.
0  images resized.
보 이미지 resize 완료!


In [5]:
# !pip install matplotlib
# !pip install scikit-learn
# !pip install scipy

In [6]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

def load_data(img_path, number_of_data=7491):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=224
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissors/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "dataset" #폴더명
(X, y)=load_data(image_dir_path)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
x_train_norm = X_train/255.0   # 입력은 0~1 사이의 값으로 정규화
x_val_norm = X_val/255.0
print("x_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 7491 입니다.
x_train shape: (5243, 224, 224, 3)
y_train shape: (5243,)


In [56]:
# for i in range(100):
#     plt.imshow(x_train[i])
#     print('라벨 : ', y_train[i])
#     plt.show()

In [57]:
# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(224,224,3)))  # 필터와 커널 크기 변경
# model.add(keras.layers.MaxPool2D(3,3))  # 풀링 크기 변경
# model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
# model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256, activation='relu'))  # Dense 레이어 유닛 증가
# model.add(keras.layers.Dense(20, activation='softmax'))

In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
# model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,035 (91.20 MB)

 Trainable params: 23,907,907 (91.20 MB)

 Non-trainable params: 128 (512.00 B)

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

optimizer = keras.optimizers.Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# ModelCheckpoint 콜백 설정
checkpoint = ModelCheckpoint(
    'dataset/model/best_model003.keras',  # 경로를 지정하여 저장
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=20,     # 이미지 회전
    width_shift_range=0.2, # 가로로 이동
    height_shift_range=0.2,# 세로로 이동
    shear_range=0.2,       # 시야각 변환
    zoom_range=0.2,        # 확대/축소
    horizontal_flip=True,  # 좌우 반전
    fill_mode='nearest'
)

# 증강된 이미지로 모델 학습
model.fit(datagen.flow(x_train_norm, y_train),
          validation_data=(x_val_norm, y_val),
          epochs=50,
          callbacks=[checkpoint])   # checkpoint 콜백 추가

Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 85s 512ms/step - accuracy: 0.5341 - loss: 3.1749 - val_accuracy: 0.4929 - val_loss: 2.9905
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 83s 502ms/step - accuracy: 0.5911 - loss: 2.8112 - val_accuracy: 0.5138 - val_loss: 2.7487
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 74s 451ms/step - accuracy: 0.6408 - loss: 2.5930 - val_accuracy: 0.7095 - val_loss: 2.4713
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 73s 446ms/step - accuracy: 0.6638 - loss: 2.4840 - val_accuracy: 0.7282 - val_loss: 2.3012
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 71s 432ms/step - accuracy: 0.7028 - loss: 2.3467 - val_accuracy: 0.8194 - val_loss: 2.1363
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 70s 428ms/step - accuracy: 0.7281 - loss: 2.2430 - val_accuracy: 0.7883 - val_loss: 2.0959
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 70s 426ms/step - accuracy: 0.7399 - loss: 2.1721 - val_accuracy: 0.8946 - val_loss: 1.9118
Epoch 8/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 70s 429ms/step - accuracy: 0.7761 - loss: 2

In [16]:
image_dir_path = "dataset/test"
(x_test, y_test)=load_data(image_dir_path,328)
x_test_norm = x_test/255.0

print("x_test shape : {}".format(x_test.shape))
print("y_test shape : {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 328 입니다.
x_test shape : (328, 224, 224, 3)
y_test shape : (328,)


In [17]:
# 저장된 최적의 모델 로드
best_model = keras.models.load_model('dataset/model/best_model003.keras')

# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

11/11 - 1s - 73ms/step - accuracy: 0.5671 - loss: 1.7619
test_loss : 1.7618767023086548
test_accuracy : 0.5670731663703918


In [18]:
# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_val_norm, y_val, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

71/71 - 4s - 60ms/step - accuracy: 0.9337 - loss: 0.7501
test_loss : 0.7500661611557007
test_accuracy : 0.9337188601493835


In [19]:
predicted_result = model.predict(x_test_norm)	# model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1)

idx=100		# 값을 변경해서 찾아보자
print('model.predict() 결과 : ', predicted_result[idx])
print('model이 추론한 가장 가능성이 높은 결과 : ', predicted_labels[idx])
print('실제 데이터의 라벨 : ', y_test[idx])

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
model.predict() 결과 :  [0.69847715 0.2677987  0.03372413]
model이 추론한 가장 가능성이 높은 결과 :  0
실제 데이터의 라벨 :  0


In [20]:
# plt.imshow(x_test[idx], cmap=plt.cm.binary)
# plt.show()

In [21]:
import random
wrong_predict_list=[]
for i, _ in enumerate(predicted_labels):
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)
        
samples = random.choices(population=wrong_predict_list, k=10) # k값을 수정하면 더 많은 값의 수를 볼 수 있다.

for n in samples:
    print("예측확률분포: " + str(predicted_result[n]))
    print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
    # plt.imshow(x_test[n], cmap=plt.cm.binary)
    print(n)
    # plt.show()

예측확률분포: [0.1036303  0.8795071  0.01686258]
라벨: 2, 예측결과: 1
269
예측확률분포: [0.7152822  0.21960855 0.06510921]
라벨: 2, 예측결과: 0
245
예측확률분포: [0.50912625 0.42557532 0.06529842]
라벨: 1, 예측결과: 0
113
예측확률분포: [0.03601804 0.30274105 0.6612409 ]
라벨: 1, 예측결과: 2
187
예측확률분포: [0.79256743 0.01477844 0.19265424]
라벨: 2, 예측결과: 0
230
예측확률분포: [0.03042917 0.14855333 0.82101744]
라벨: 1, 예측결과: 2
189
예측확률분포: [0.34386015 0.61473423 0.04140567]
라벨: 0, 예측결과: 1
85
예측확률분포: [0.21846426 0.7727508  0.00878491]
라벨: 0, 예측결과: 1
13
예측확률분포: [0.01948385 0.8972595  0.08325668]
라벨: 2, 예측결과: 1
310
예측확률분포: [0.25707355 0.6298617  0.11306471]
라벨: 2, 예측결과: 1
241


In [65]:
model.save('dataset/model/model.keras', include_optimizer=False)